In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
from datasets import load_dataset, Audio
torch.manual_seed(42)

from typing import Dict
from IPython.display import Audio as play_audio

2025-02-01 15:39:55.142354: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738413595.159818  701015 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738413595.169097  701015 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-01 15:39:55.187261: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
llm_name = "t-tech/T-lite-it-1.0"
tokenizer = AutoTokenizer.from_pretrained(llm_name)

In [3]:
llm = AutoModelForCausalLM.from_pretrained(
    llm_name, 
    torch_dtype=torch.bfloat16,
    device_map="balanced",
    max_memory={0: '10GB', 1: '10GB', 2: '10GB'}
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [33]:
prompt = \
'''
Задача: Классифицируй текстовые команды пилота/бортового ИИ в один из 9 классов на основе их содержания. В ответе не давай обоснования и названия класса, а ТОЛЬКО НОМЕР.

Примеры:

1. Запрос: "Опустись на уровень 4.2 километра"  
   Класс: 1 (Снизить высоту)

2. Запрос: "Набери 7.5 км высоты"  
   Класс: 2 (Увеличить высоту)

3. Запрос: "Тормози до 300 км/ч"  
   Класс: 3 (Снизить скорость)

4. Запрос: "Ускорение до 650 км/час"  
   Класс: 4 (Увеличить скорость)

5. Запрос: "Подготовить закрылки и шасси"  
   Класс: 5 (Запустить подготовку к посадке)

6. Запрос: "Проверить топливные системы перед стартом"  
   Класс: 6 (Начать проверку систем перед взлетом)

7. Запрос: "Полный отчет о работоспособности"  
   Класс: 7 (Проверить состояние всех систем)

8. Запрос: "Вывести данные о воздушной скорости"  
   Класс: 8 (Отобразить текущую скорость)

9. Запрос: "Активировать карту маршрута"  
   Класс: 9 (Открыть систему навигации)

---

Новый запрос для классификации: "{command_text}"  
Класс (номер):
'''

In [34]:
def text_classify(text: str) -> str:

    messages = [
        {"role": "system", "content": "Твоя задача - быть полезным диалоговым ассистентом."},
        {"role": "user", "content": prompt.replace('{command_text}', text)}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    model_inputs = tokenizer([text], return_tensors="pt").to(llm.device)
    
    generated_ids = llm.generate(
        **model_inputs,
        max_new_tokens=24,
        do_sample=False
    )
    
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

In [35]:
text_classify('Увеличить скорость до 960 км/ч')

'4'

In [28]:
speech_to_text_model_name = "NLPVladimir/wav2vec2-large-xls-r-300m-ru"  

processor = Wav2Vec2Processor.from_pretrained(
    speech_to_text_model_name,
    device='cuda:0'
)

speech_to_text_model = Wav2Vec2ForCTC.from_pretrained(
    speech_to_text_model_name,
    device_map='cuda:0',
)

In [29]:
def get_text_from_raw_speech(audio: Dict) -> str:
    input_values = processor(
        audio['array'], 
        sampling_rate=audio['sampling_rate'], 
        return_tensors="pt"
    ).input_values.to(speech_to_text_model.device)
    
    with torch.no_grad():
        logits = speech_to_text_model(input_values).logits
    
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]

    return transcription

In [9]:
dataset = load_dataset("NLPVladimir/aviation-command-dataset-ru")

In [10]:
audio_example = dataset['train'][0]['audio']

In [11]:
play_audio(audio_example['array'], rate=audio_example['sampling_rate'])

In [30]:
get_text_from_raw_speech(audio_example)

'проверить состояние двигатили бех зем'

In [36]:
def classify_command_from_speech_raw(audio: Dict):
    
    command_text = get_text_from_raw_speech(audio)
    predicted_class = text_classify(command_text)

    return predicted_class

In [37]:
classify_command_from_speech_raw(audio_example)

'6'

In [48]:
def add_raw_answer(batch):
    batch["raw_answer"] = classify_command_from_speech_raw(batch['audio'])
    return batch

In [ ]:
dataset['train'] = dataset['train'].map(add_raw_answer)

Map:   0%|          | 0/1182 [00:00<?, ? examples/s]